In [ ]:
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from api_util import read_token, BASE_URL, get_headers
import requests


token = read_token()

def _make_get_request(endpoint, params=None, quiet=None):
    """共用的 HTTP GET 請求處理函數"""
    try:
        response = requests.get(f"{BASE_URL}/{endpoint}", headers=get_headers(token), params=params)
        response.raise_for_status()
        result = response.json()
        if not quiet: 
            print(result)
        return result
    except requests.exceptions.RequestException as err:
        print(f"Request failed: {err}")
        return None
    
def _make_post_request(endpoint, data, quiet=None):
	try:
		response = requests.post(f"{BASE_URL}/{endpoint}", headers=get_headers(token), json=data)
		response.raise_for_status()
		result = response.json()
		if not quiet: 
			print(result)
		return result
	except requests.exceptions.RequestException as err:
		print(f"Request failed: {err}")
		return None
    
def get_all_technews (page, size, keyword):
	res =  _make_get_request('technews',  params={
		'page': page,
		'size': size,
        'keyword': keyword
	})
	return res.get('data')[::-1]
    
def create_one_company_news (params):
    res = _make_post_request('/company-news', params)
    return res
    
def get_all_company_news (page, size):
	res =  _make_get_request('company-news', {
		'page': page,
		'size': size
	})
	return res.get('data')[::-1]
    
def get_all_company_news_by_keyword (keyword, page=1, size=10):
	res =  _make_get_request('/company-news/search', {
		'page': page,
		'size': size,
		'keyword': keyword
	})
	return res.get('data')

def get_user_favorite_news():
    res = _make_get_request('userFavorite/technews')
    return res.get('data')
    
def user_subscribe_news(params):
    res = _make_post_request('/userFavorite/technews', params)
    return res

def print_news(news):
    for item in news:
        company_name = item.get('companyName', '')
        news_id = item.get('id', '--')
        title = item.get('title', '--')
        release_time = item.get('release_time', '--')
        web_url = item.get('web_url', '--')
        print(company_name)
        print(f"{news_id} : {title} {release_time} \n")
        if web_url:
            print(web_url)
        print('----------')

In [9]:
keyword = 'ai'

news = get_all_technews(1, 10, keyword=keyword)

for item in news:
	id = item.get('id', '--')
	title = item.get('title', '--')
	print(f"{id} : {title} \n")

{'code': 200, 'message': '成功', 'data': [{'id': 124143, 'title': '與競爭對手差距恐擴大，郭明錤籲蘋果加大 AI 投資', 'release_time': '2025-06-10T14:15:00.000Z', 'publisher': 'MoneyDJ', 'web_url': 'https://technews.tw/2025/06/10/wwdc-2025-ai-apple/', 'createdAt': '2025-06-10T06:28:40.000Z', 'updatedAt': '2025-06-10T06:28:40.000Z'}, {'id': 124134, 'title': '美國大學搶先布局，AI 相關課程成新教學主流', 'release_time': '2025-06-10T11:45:00.000Z', 'publisher': 'TechNews 編輯台', 'web_url': 'https://technews.tw/2025/06/10/university-going-to-add-some-ai-course-in-the-system/', 'createdAt': '2025-06-10T06:28:40.000Z', 'updatedAt': '2025-06-10T06:28:40.000Z'}, {'id': 124126, 'title': 'ChatGPT 推動迅速成長，OpenAI 年度經常性收入破 100 億美元大關', 'release_time': '2025-06-10T10:49:00.000Z', 'publisher': '陳 冠榮', 'web_url': 'https://finance.technews.tw/2025/06/10/openai-hits-10-billion-in-annual-recurring-revenue/', 'createdAt': '2025-06-10T06:28:30.000Z', 'updatedAt': '2025-06-10T06:28:30.000Z'}, {'id': 124125, 'title': '黃仁勳：AI 推動人類語言取代程式語言，人人都能成為工程師', 'release

In [ ]:
all_news = get_all_technews(page=1, size=100)
for item in all_news:
	id = item.get('id', '--')
	title = item.get('title', '--')
	print(f"{id} : {title} \n")

In [ ]:
all_news = get_all_company_news(page=1, size=100)

print_news(all_news)

In [ ]:
keyword = 'ai'
search_news = get_all_company_news_by_keyword(page=1, size=100, keyword=keyword)

print_news(search_news)

In [ ]:
symbol = 'tsla'

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime, timezone

with open('index.html', 'r', encoding='utf-8') as file:
	html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

news = []

for row in soup.find_all('tr', class_='cursor-pointer has-label'):
	time_cell = row.find('td', align='right')
	time_text = time_cell.get_text(strip=True).replace('Today', '')

	if 'Today' in time_text:
		time_text = time_text.replace('Today', '')
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()
	else:
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()

	try:
		time_obj = datetime.strptime(time_text, '%I:%M%p')
		time_24hr = time_obj.strftime('%H:%M:%S')
	except ValueError:
		# 如果时间格式无法解析，使用默认时间
		time_24hr = '00:00:00'
		
	# 创建 MySQL 支持的时间字符串
	release_time = f"{date_str} {time_24hr}"

	link = row.find('a', class_='tab-link-news')
	title = link.get_text(strip=True)
	web_url = link['href']

	publisher = row.find('span').get_text(strip=True).strip('()')

	news_item = {
        "title": title,
        "symbol": symbol.upper(),
        "release_time": release_time,
        "publisher": publisher,
        "web_url": web_url
    }

	news.append(news_item)

print(json.dumps(news[0:2], indent=4))


In [ ]:
from time import sleep

SLEEP_SECOND = 1/ 10
for item in news:
	res = create_one_company_news(item)
	print(res)
	sleep(SLEEP_SECOND)

In [ ]:
newsId = 123262
data = user_subscribe_news({'newsId': newsId})
data

In [ ]:
data = get_user_favorite_news()
data